### Spark Initialization

In [1]:
import findspark
findspark.init()

In [2]:
import sys
import copy
import csv
import numpy as np

from string import atoi
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
conf = SparkConf().setAppName("ContentBased")
conf = conf.setMaster("local[*]")

In [4]:
sc  = SparkContext(conf=conf)

### Load Train and Test Data

In [5]:
trainData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_train.txt",use_unicode=False)
testData = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pittsburgh_review_with_text_20_res_lemma_data_test.txt",use_unicode=False)

In [6]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

Calculate average rating of users and products

In [7]:
avg_rating = train_rdd.map(lambda x: (x[0][0], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))
prod_rating = train_rdd.map(lambda x: (x[0][1], x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).map(lambda x: (x[0], sum(x[1])/len(x[1])))

In [8]:
train_temp = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), x[2]))

### Load product category data

In [9]:
data = sc.textFile("/Users/lakshya/Desktop/INF-553/Project/pitts_bus_20_attributes_categories.txt",use_unicode=False)

In [10]:
data.take(6)

['"f2FfutZhb4F-m1Ob0EdYaw","asian fusion caterers fast food chinese restaurants vegetarian food delivery services event planning  services food  alcohol_full_bar ambience_casual bikeparking businessacceptscreditcards businessparking_street caters goodforkids goodformeal_dinner hastv noiselevel_average restaurantsattire_casual restaurantsdelivery restaurantsgoodforgroups restaurantspricerange2_2 restaurantstableservice restaurantstakeout wheelchairaccessible "',
 '"qfcdMhm1Ff28JHVpHca20g","pizza restaurants  businessacceptscreditcards restaurantsattire_casual restaurantspricerange2_2 restaurantstakeout "',
 '"wHUru-79ExNRanJtwZEYeA",""',
 '"rreIYrzI9U052p7nN_qogA","sushi bars food thai poke japanese restaurants   ambience_casual bikeparking businessacceptscreditcards caters goodforkids goodformeal_lunch goodformeal_dinner noiselevel_average restaurantsattire_casual restaurantspricerange2_2 restaurantstakeout "',
 '"eva56motCJcevOwKzyQO1g","cafes hotels  travel swimming pools travel serv

In [11]:
train_data = data.mapPartitions(lambda x: csv.reader(x)).map(lambda x: (x[0], x[1]))

In [12]:
train_data.take(3)

[('f2FfutZhb4F-m1Ob0EdYaw',
  'asian fusion caterers fast food chinese restaurants vegetarian food delivery services event planning  services food  alcohol_full_bar ambience_casual bikeparking businessacceptscreditcards businessparking_street caters goodforkids goodformeal_dinner hastv noiselevel_average restaurantsattire_casual restaurantsdelivery restaurantsgoodforgroups restaurantspricerange2_2 restaurantstableservice restaurantstakeout wheelchairaccessible '),
 ('qfcdMhm1Ff28JHVpHca20g',
  'pizza restaurants  businessacceptscreditcards restaurantsattire_casual restaurantspricerange2_2 restaurantstakeout '),
 ('wHUru-79ExNRanJtwZEYeA', '')]

### Collect user data from train data (User, Product, Rating)

In [13]:
userReview = train_data.map(lambda x: (x[0][0], x[0][1], x[1][0][0]))

In [14]:
userReviewCollected = train_temp.map(lambda x: (x[0][0], x[0][1], x[1]))

In [15]:
userReviewCollected.take(5)

[('1VVHf1BvtGC0aSCCIjQyiA', 'K5jY2W5Q3eNnwssV5UZtow', '4'),
 ('QYKexxaOJQlseGWmc6soRg', 'rzByiKaj-bLeLz-zKNBQdg', '2'),
 ('-ARdx8hOcEWlMDjzwLYZ_g', '3cbsPfoUUrysf-M8FI_0IA', '4'),
 ('2oTG2GrBpikkonRxQywI7Q', 'CFtZH4Skp9z3o4ToSywI4w', '4'),
 ('B7aR8VlLMGzaJ7NHC-jeMQ', 'RC8NJlX3X2Eo240PU5zJWA', '4')]

### Collect product data (Product, Category Text)

In [16]:
prodReview = train_data.map(lambda x: (x[0], x[1])).groupByKey().mapValues(list)

In [17]:
prodReviewCollected = prodReview.map(lambda x: (x[0], x[1][0].split(" ")))

In [18]:
prodReviewCollected.take(5)

[('MvlQo4bev1eqp1q0HYOLHg',
  ['professional',
   'services',
   'performing',
   'arts',
   'arts',
   '',
   'entertainment',
   'fitness',
   '',
   'instruction',
   'dance',
   'studios',
   'active',
   'life',
   'education',
   '',
   'bikeparking',
   'businessparking_street',
   'byappointmentonly',
   'goodforkids',
   'wheelchairaccessible',
   '']),
 ('jemRU9DvmS0WGZuZ-k3jCQ',
  ['desserts',
   'food',
   'shaved',
   'ice',
   'ice',
   'cream',
   '',
   'frozen',
   'yogurt',
   '',
   'bikeparking',
   'businessacceptscreditcards',
   'businessparking_lot',
   'caters',
   'restaurantspricerange2_1',
   'restaurantstakeout',
   '']),
 ('PuMpFTKS6gY_e31UP5YVnw',
  ['restaurants',
   'american_new',
   '',
   'bikeparking',
   'businessacceptscreditcards',
   'goodforkids',
   'hastv',
   'restaurantsattire_casual',
   'restaurantsgoodforgroups',
   'restaurantspricerange2_1',
   'restaurantstakeout',
   '']),
 ('5S8KaaAIjqo2bJcVlMNW5w',
  ['chinese',
   'restaurants',
 

### Doc2Vec model creation

In [19]:
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

/Users/lakshya/anaconda/lib/python2.7/site-packages/cffi/model.py:525: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


Convert user and product rdd to pandas dataframe

In [20]:
spark = SparkSession(sc)

userPandas = userReviewCollected.toDF().toPandas()
prodPandas = prodReviewCollected.toDF().toPandas()

Create Doc2Vec model on product category text

In [21]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(prodPandas['_2'])]

In [22]:
documents

[TaggedDocument(words=[u'professional', u'services', u'performing', u'arts', u'arts', u'', u'entertainment', u'fitness', u'', u'instruction', u'dance', u'studios', u'active', u'life', u'education', u'', u'bikeparking', u'businessparking_street', u'byappointmentonly', u'goodforkids', u'wheelchairaccessible', u''], tags=[0]),
 TaggedDocument(words=[u'desserts', u'food', u'shaved', u'ice', u'ice', u'cream', u'', u'frozen', u'yogurt', u'', u'bikeparking', u'businessacceptscreditcards', u'businessparking_lot', u'caters', u'restaurantspricerange2_1', u'restaurantstakeout', u''], tags=[1]),
 TaggedDocument(words=[u'restaurants', u'american_new', u'', u'bikeparking', u'businessacceptscreditcards', u'goodforkids', u'hastv', u'restaurantsattire_casual', u'restaurantsgoodforgroups', u'restaurantspricerange2_1', u'restaurantstakeout', u''], tags=[2]),
 TaggedDocument(words=[u'chinese', u'restaurants', u'', u'businessacceptscreditcards', u'restaurantsattire_casual', u'restaurantsdelivery', u'restau

In [23]:
model = Doc2Vec(documents, size = 2000, min_count = 1, workers=4)

/Users/lakshya/anaconda/lib/python2.7/site-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [24]:
feature_vectors = []
for i in range(0, len(model.docvecs)):
    feature_vectors.append(model.docvecs[i])

In [25]:
feature_vectors[0]

array([-0.00085623,  0.00538157, -0.01083877, ...,  0.00503241,
        0.00432282,  0.00195231], dtype=float32)

Add Doc2Vec vectors to product dataframe

In [26]:
prodPandas['Vector'] = feature_vectors

In [27]:
prodPandas

,_1,_2,Vector
0,MvlQo4bev1eqp1q0HYOLHg,"[professional, services, performing, arts, art...","[-0.000856232, 0.00538157, -0.0108388, 0.00452..."
1,jemRU9DvmS0WGZuZ-k3jCQ,"[desserts, food, shaved, ice, ice, cream, , fr...","[-2.89002e-05, 0.000403871, -0.00141715, 0.000..."
2,PuMpFTKS6gY_e31UP5YVnw,"[restaurants, american_new, , bikeparking, bus...","[-0.000163175, 0.00180052, -0.0043844, 0.00192..."
3,5S8KaaAIjqo2bJcVlMNW5w,"[chinese, restaurants, , businessacceptscredit...","[0.000363107, 0.00188846, -0.00416163, 0.00189..."
4,xZfPdAYeimiruXuGR4nSUA,"[specialty, food, health, markets, vegetarian,...","[-0.000158673, 0.00144021, -0.00264549, 0.0011..."
5,8A4ck_lvMLYZDrDt7w8GVA,[],"[7.38954e-05, -0.000146097, 0.0002251, 4.58524..."
6,P2uPCEBP2VZsApnvEHCgag,"[indian, restaurants, , , ambience_casual, bik...","[0.000347396, -0.00047828, 0.00065993, 0.00031..."
7,DwmAsemahJywywhuqv5J1Q,"[szechuan, restaurants, chinese, , , ambience_...","[0.000477173, 0.00033229, -0.00147382, 0.00099..."
8,71ONxJtmDNDiJAtYHK-0RA,"[american_traditional, restaurants, diners, br...","[-8.49649e-05, 0.00131307, -0.00303418, 0.0015..."
9,_7uEyBADd8S14rp2Kzez5g,"[bars, nightlife, dive, bars, beer, bar, , alc...","[-0.000241363, 0.000776163, -0.00173951, 0.000..."


Product category text not needed

In [28]:
del prodPandas['_2']

### Weighted Linear Combination of product vectors for users

In [29]:
userPandas['Vector'] = [[] for _ in range(len(userPandas))]

In [30]:
userPandas['_3'] = userPandas['_3'].astype(float)

In [31]:
for index, row in userPandas.iterrows():
    vector = np.array(prodPandas.loc[prodPandas['_1'] == row['_2'], 'Vector'].values[0])
    rating = row['_3']
    userPandas.at[index,'Vector'] = rating*vector

In [32]:
userPandas

,_1,_2,_3,Vector
0,1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4.0,"[-0.00242437, 0.00754665, -0.0167179, 0.002002..."
1,QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2.0,"[-0.000144155, 0.00213548, -0.0044292, 0.00206..."
2,-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4.0,"[-0.000100993, 0.00992949, -0.022101, 0.009570..."
3,2oTG2GrBpikkonRxQywI7Q,CFtZH4Skp9z3o4ToSywI4w,4.0,"[0.000914383, -0.000350836, 0.000166445, -0.00..."
4,B7aR8VlLMGzaJ7NHC-jeMQ,RC8NJlX3X2Eo240PU5zJWA,4.0,"[-0.00165324, 0.00736487, -0.0145778, 0.005350..."
5,UsmTxWbobLsI6WR1Db0W7A,hsDkxhc-_YYsVqAeQBJJjg,1.0,"[0.000414565, -0.0018823, 0.00346436, -0.00137..."
6,2jKzO_01d12oiu-2bOYcYg,5REYrZfsX3m4E3FTwovp5Q,4.0,"[-0.00066343, 0.0314426, -0.0680979, 0.0304236..."
7,57tf_RmT5o3BiFj2gpV-Jg,mc5Ha4QyQZvU7fzr-ubhhw,5.0,"[-0.00173485, 0.0188769, -0.0431066, 0.0159233..."
8,J9OSJ7Xl9wnS0tL709AIqA,dLc1d1zwd1Teu2QED5TmlA,4.0,"[0.00201334, 0.00834269, -0.0175282, 0.0120538..."
9,BTg_ajmMJKKpvH0P1mAUqA,oxig-TT9V2QttAvRqa5WNA,3.0,"[-0.00121105, 0.0102232, -0.018733, 0.0066356,..."


Product and rating column not needed

In [33]:
del userPandas['_2']
del userPandas['_3']

Linear combination of feature vectors

In [34]:
userPandas = userPandas.groupby(['_1']).sum()

Normalize the user feature vectors

In [35]:
from sklearn.preprocessing import Normalizer

/Users/lakshya/anaconda/lib/python2.7/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


In [36]:
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector']).reshape(1, -1)
    transformer = Normalizer().fit(vector)
    userPandas.at[index,'Vector'] = transformer.transform(vector)

In [37]:
userPandas

,Vector
_1,
-0-hVEpwWEcJLJoGq3rE3g,"[[-0.00245415, 0.0305672, -0.0653684, 0.028703..."
-2OB54nQ6FsGLUM-R1KXnA,"[[-0.000673377, 0.0304585, -0.064962, 0.029555..."
-ARdx8hOcEWlMDjzwLYZ_g,"[[-0.00203685, 0.0304457, -0.0655553, 0.028676..."
-Pk25bOBsvemFaWKDBVBzA,"[[-0.00168887, 0.0288449, -0.063727, 0.0295085..."
-Q2wBtscwW6JOqlBndji4A,"[[-0.00139252, 0.03028, -0.0642732, 0.0291131,..."
-Q4bjWlbxmb1yKP4U7OODg,"[[-0.00230569, 0.030761, -0.0644459, 0.0291208..."
-SDx-d5jppC4OBBosLVpYw,"[[0.000319544, 0.0292179, -0.064431, 0.0308398..."
-XgVXGJnOnW0kQEol6O3Pg,"[[-0.00222631, 0.0305794, -0.0658329, 0.028066..."
-Y6tXYPYqeVy37-L5p0rMw,"[[-0.00158423, 0.0302251, -0.0655342, 0.028769..."


### Create user numpy matrix from feature vectors

In [38]:
user_matrix = np.zeros((len(userPandas), len(feature_vectors[0])))
idx = 0
for index, row in userPandas.iterrows():
    vector = np.array(row['Vector'])[0]
    user_matrix[idx] = vector
    idx += 1
    

In [39]:
user_matrix

array([[-0.00245415,  0.03056724, -0.06536841, ...,  0.03531742,
         0.02023615,  0.01173127],
       [-0.00067338,  0.0304585 , -0.06496196, ...,  0.03746143,
         0.01873488,  0.0129635 ],
       [-0.00203685,  0.0304457 , -0.06555529, ...,  0.03631432,
         0.02002098,  0.01249701],
       ..., 
       [-0.00099086,  0.03034398, -0.0646295 , ...,  0.03657493,
         0.01936147,  0.01279344],
       [-0.00093954,  0.03021389, -0.06367598, ...,  0.03595189,
         0.0209174 ,  0.01259169],
       [-0.00130096,  0.0293319 , -0.06632555, ...,  0.03771533,
         0.01979822,  0.01212249]])

### Create product numpy matrix from feature vectors

In [40]:
prod_matrix = np.zeros((len(prodPandas), len(feature_vectors[0])))
idx = 0
for index, row in prodPandas.iterrows():
    vector = np.array(row['Vector'])
    prod_matrix[idx] = vector
    idx += 1

In [41]:
prod_matrix

array([[ -8.56231956e-04,   5.38157299e-03,  -1.08387684e-02, ...,
          5.03240945e-03,   4.32281988e-03,   1.95230707e-03],
       [ -2.89001764e-05,   4.03870712e-04,  -1.41715200e-03, ...,
          3.96555522e-04,   9.71130969e-04,   7.93375875e-05],
       [ -1.63175224e-04,   1.80052058e-03,  -4.38439846e-03, ...,
          2.38867849e-03,   1.59371505e-03,   8.70974327e-04],
       ..., 
       [  3.87326363e-05,   1.73313823e-03,  -3.76818259e-03, ...,
          2.37870100e-03,   8.65979062e-04,   7.98312365e-04],
       [  1.84004639e-05,   4.14141599e-04,  -6.32672687e-04, ...,
          4.20163851e-04,   5.75383456e-06,   1.51280532e-04],
       [ -2.76877458e-04,   3.24457232e-03,  -7.07618147e-03, ...,
          3.53280222e-03,   2.35253829e-03,   1.37208391e-03]])

### Compute cosine similarity by taking dot product

In [42]:
similarity_matrix = np.dot(user_matrix, prod_matrix.T)

In [43]:
similarity_matrix.shape

(987, 3192)

### Flatten similarity matrix to relate with user and products

In [44]:
prod = prodPandas['_1'].values
user = userPandas.index.values

In [45]:
zf = similarity_matrix.flatten()
xr = np.repeat(user, prod.size)
yt = np.tile(prod, user.size)
d = np.stack((xr, yt, zf), axis=-1)

In [46]:
d.shape

(3150504, 3)

### Convert similarity matrix to RDD

In [47]:
similarity_rdd = sc.parallelize(d)

In [48]:
similarity_rdd.take(5)

[array([u'-0-hVEpwWEcJLJoGq3rE3g', u'MvlQo4bev1eqp1q0HYOLHg',
        0.17232228536323602], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'jemRU9DvmS0WGZuZ-k3jCQ',
        0.018671802356957642], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'PuMpFTKS6gY_e31UP5YVnw',
        0.0672202220904829], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'5S8KaaAIjqo2bJcVlMNW5w',
        0.06444113264958053], dtype=object),
 array([u'-0-hVEpwWEcJLJoGq3rE3g', u'xZfPdAYeimiruXuGR4nSUA',
        0.04287442882346724], dtype=object)]

### Use Train and Test Data for predictions

In [49]:
trainData.take(5)

['1VVHf1BvtGC0aSCCIjQyiA,K5jY2W5Q3eNnwssV5UZtow,4,2016-11-16,2,2,2,past sunday one several time ive spirit its always eccentric fun time first music performance second hang recent visit sundays bingo bango spirit know pizza good drink also awesome last time get chard margarita time get tomatillo margarita hot ciders its always pleasant surprise see whats menu food drink drink little pricey drawback one coolest things spirit atmosphere its always super strange positive way really never know expect bingo bango definitely family appropriate its fun activity friends maybe even date doesnt mind something ordinary',
 'QYKexxaOJQlseGWmc6soRg,rzByiKaj-bLeLz-zKNBQdg,2,2015-04-13,0,0,0,old cramp build lot enough employees staff keep demand cause long wait time',
 '-ARdx8hOcEWlMDjzwLYZ_g,3cbsPfoUUrysf-M8FI_0IA,4,2014-03-24,6,4,3,live long world without donut menu dont know group nine din three varieties donuts include lemon lavender chocolate espresso zeppolli amaze pepper donut concoction ever m

Create key on (user, product)

In [50]:
train_rdd = trainData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))
test_rdd = testData.mapPartitions(lambda x: csv.reader(x)).map(lambda x: ((x[0], x[1]), float(x[2])))

In [51]:
similarity_rdd = similarity_rdd.map(lambda x: ((x[0], x[1]), float(x[2])))

In [52]:
similarity_rdd.take(5)

[((u'-0-hVEpwWEcJLJoGq3rE3g', u'MvlQo4bev1eqp1q0HYOLHg'), 0.17232228536323602),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'jemRU9DvmS0WGZuZ-k3jCQ'),
  0.018671802356957642),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'PuMpFTKS6gY_e31UP5YVnw'), 0.0672202220904829),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'5S8KaaAIjqo2bJcVlMNW5w'), 0.06444113264958053),
 ((u'-0-hVEpwWEcJLJoGq3rE3g', u'xZfPdAYeimiruXuGR4nSUA'), 0.04287442882346724)]

Join similarity matrix with train and test RDD to take only similarity values for training and testing

In [53]:
train = similarity_rdd.join(train_rdd)
test = similarity_rdd.join(test_rdd)

In [55]:
test.take(5)

[((u'nzl2KyGKLtV7j8QEfOhC-w', u'TU2oECyx4VhCnzRzGzOlTg'),
  (0.027502951296124227, 4.0)),
 ((u'JeIVRtZiwUnSP9qR4AmfEQ', u'WZDQqw960DaMWDyPbtRFtA'),
  (-0.08572004246191142, 4.0)),
 ((u'g3V76Ja0XgWS1rqx0gxL_A', u'a384anuGRsOe5IXclb3cNQ'),
  (0.02305033882924474, 4.0)),
 ((u'3FjdHwnoz-vfw6hBBtckqg', u'cNCXUYQd3kFR_AyfjTejQQ'),
  (0.09674220146375062, 4.0)),
 ((u'3DltS5Wr9MOntnc8StBFBA', u'GjX2WgchvvS-JAWBmzkLyQ'),
  (0.11785955303588883, 5.0))]

Convert RDD to List

In [56]:
train_ratings = train.collect()
test_ratings = test.collect()

In [57]:
train_ratings

[((u'hHqH_E9FCI_B6WubV0jPYA', u'ZNdV9ytExuxPTXSN8i2xhw'),
  (0.1916465316364808, 3.0)),
 ((u'V0pP_PQnWdtyKpF-pifiaw', u'Fpm3WvqtrAg2ueh_4pz7iA'),
  (0.24962467396670976, 4.0)),
 ((u'IKnLl7SbuP0u6HS34jwHhw', u'guQww9yGHP7rRTea6zTnDg'),
  (0.21229800220422576, 3.0)),
 ((u'4r33dXcE1oYZxjONrhxTiA', u'9gNko6cFCMZbvy1zhJ7-Xg'),
  (0.03790061045685463, 5.0)),
 ((u'Rem81Xoev05aqeA-mFbM4A', u'1LUaZFVMEjodl1tbAGF3sQ'),
  (0.011718898038154223, 4.0)),
 ((u'4wp4XI9AxKNqJima-xahlg', u'nc5uuDeM3EA9WJycGDeg1w'),
  (0.10380067542198751, 4.0)),
 ((u'BkMqpJikNc3r5itc-ui6ww', u'x8WI_GkeGHGJCXggDm8flg'),
  (0.1220257429513033, 3.0)),
 ((u'ftUrNSpxUr8eWCyOlsZVgg', u'-zu_7iWh5CnV9kEQyCDAQQ'),
  (0.14993426446311797, 4.0)),
 ((u'BkMqpJikNc3r5itc-ui6ww', u'Tdc2v9DBRRYeG2P_h9OvRw'),
  (0.00905688589622861, 3.0)),
 ((u'1ZPnQs-tdvbX8ROjtnzcEg', u'OXrFWgoz533T8tMRemkiww'),
  (0.11235011632190497, 5.0)),
 ((u'tR12WeWha2DGhUrKcvTttQ', u'T-4MZRvKlF3VLDcjCqqFEg'),
  (0.016919765404272126, 5.0)),
 ((u'b4ZbEEDbCPT6pVT4

### Training Regressor on similarity values

In [58]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

Convert data to numpy array for regressor

In [59]:
X_train = []
y_train = []
for ratings in train_ratings:
    X_train.append(ratings[1][0])
    y_train.append(ratings[1][1])

X_train = np.array(X_train)
X_train = X_train.reshape(-1,1)

In [60]:
X_test = []
y_test = []
for ratings in test_ratings:
    X_test.append(ratings[1][0])
    y_test.append(ratings[1][1])

X_test = np.array(X_test)
X_test = X_test.reshape(-1,1)

In [61]:
X_test.shape

(10002, 1)

Train regressor on training data and make prediction on Test data<br>
Computed Mean Squared Error on predicted values

In [65]:
forest = RandomForestRegressor(max_depth=8, n_estimators=20)

rs = GradientBoostingRegressor(loss='ls', learning_rate=0.005, n_estimators=10000)

reg = LinearRegression()

rs.fit(X_train, y_train)

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

expected = y_test
predicted = rs.predict(X_test)

train_expected = y_train
train_predicted = rs.predict(X_train)

print("Training:\n%s" % np.sqrt(mean_squared_error(train_expected, train_predicted)))
print("Result:\n%s" % np.sqrt(mean_squared_error(expected, predicted)))

Training:
0.99518396347
Result:
1.02815442246


### Computed Ratings on missing values

In [66]:
missing_test = test_rdd.subtractByKey(test)

In [67]:
missing_ratings_user = missing_test.map(lambda x: ((x[0][0]), (x[0][1], x[1]))).join(avg_rating).map(lambda x: ((x[0], x[1][0][0]), (x[1][1], x[1][0][1])))

In [68]:
missing_ratings_user.take(5)

[]

Make predictions on test data using the trained regressor

In [69]:
predictions = test.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [70]:
final_predictions = predictions.union(missing_ratings_user)

In [73]:
final_predictions.count()

12356

In [74]:
mse = final_predictions.map(lambda x: (x[1][0]-x[1][1])**2)

In [75]:
np.sqrt(mse.mean())

1.0243836702939397

In [71]:
final_predictions = final_predictions.collect()

### Save predictions file

In [72]:
with open('/Users/lakshya/Desktop/INF-553/Project/Doc2Vec_CategoryBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in final_predictions:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")

### Making predictions on training data for Hybrid model

In [73]:
train_predict = train.map(lambda x: ((x[0]), (rs.predict(np.array(x[1][0]).reshape(1,-1))[0], x[1][1])))

In [74]:
train_predict.take(5)

[((u'hHqH_E9FCI_B6WubV0jPYA', u'ZNdV9ytExuxPTXSN8i2xhw'),
  (3.7550129729655866, 3.0)),
 ((u'V0pP_PQnWdtyKpF-pifiaw', u'Fpm3WvqtrAg2ueh_4pz7iA'),
  (4.1039943778403458, 4.0)),
 ((u'IKnLl7SbuP0u6HS34jwHhw', u'guQww9yGHP7rRTea6zTnDg'),
  (4.1952673960789406, 3.0)),
 ((u'4r33dXcE1oYZxjONrhxTiA', u'9gNko6cFCMZbvy1zhJ7-Xg'),
  (3.7982065041427675, 5.0)),
 ((u'Rem81Xoev05aqeA-mFbM4A', u'1LUaZFVMEjodl1tbAGF3sQ'),
  (3.6527998084084974, 4.0))]

In [166]:
train_predictions = train_predict.map(lambda x: (x[1][0]-x[1][1])**2)

In [167]:
np.sqrt(train_predictions.mean())

1.03320310263514

In [76]:
train_predict = train_predict.collect()

In [77]:
with open('/Users/lakshya/Desktop/INF-553/Project/TrainDoc2Vec_CategoryBasedPredictions_pittsburgh.txt', 'w') as f:
    for item in train_predict:
        f.write(str(item[0][0])+","+str(item[0][1])+","+str(item[1][0])+"\n")